# 条件随机场 CRF

## 概率无向图模型

回顾一下之前讲解的概率无向图模型：https://applenob.github.io/graph_model.html

总结一下：

- **最大团**：无向图$G$中任何两个结点都有边连接的结点子集称为团（clique）。若团$C$不能在加入任何一个结点使其称为一个更大的团，则称$C$为图$G$的一个最大团。
- 概率无向图模型的**联合概率分**布可以表示成其最大团上的随机变量的函数的乘积形式。这也被称为概率无向图模型的**因子分解**。
- $P(Y) = \frac{1}{Z}\prod_C\psi_C(Y_C)$，其中，$Z$是规范化因子，$Z = \sum_Y\prod_C\psi_C(Y_C)$，$\psi_C(Y_C)$称为**势函数**，要求势函数是严格正的（因为涉及到累乘）。

## 条件随机场

条件随机场（Conditional Random Field， CRF）也是一种无向图模型。它是在给定随机变量$X$的条件下，随机变量$Y$的马尔科夫随机场。